################################################################################
# 1. Preparations

In [1]:
%%capture
%pip install levenshtein

In [2]:
%load_ext lab_black

In [3]:
QPATH = "Quantlet/3-data-preprocessing"

In [4]:
# PREPARE WORKING DIRECTORY

import sys

IN_COLAB = "google.colab" in sys.modules

import os

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )
else:
    os.chdir("./")

# sys.path.append('../src')

In [5]:
# PACKAGES

import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

import nltk

nltk.download("punkt")

from tqdm import tqdm

tqdm.pandas()

import importlib
import preprocessing_utils

importlib.reload(preprocessing_utils)
from preprocessing_utils import *

from sklearn.model_selection import train_test_split
from Levenshtein import distance

# SETTINGS

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option("display.max_colwidth", 500)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Processing

In [45]:
DATE = "20231021"
RS = 111

In [46]:
with open(
    f"../../data/preprocessed/Quantlet/20231014/Quantlets_20231014.pkl", "rb"
) as file:
    df = pickle.load(file)

In [47]:
df_long = df_metainfo_parse(df=df, prepare_script=True, remove_other=True)

(4837, 6)
(4836, 12)


In [48]:
df_long = clean_up(df_long)
print(df_long.shape)

100%|██████████| 4836/4836 [00:00<00:00, 673226.93it/s]

(4828, 15)


In [49]:
df_long.code_script

0       close all\ngraph = 1;\n% Open & read txt file with names of files to work with\nfilelist = 'filelist.txt';\nfilenames = textread(filelist,'%s');\nnfiles = length(filenames);\ndisp([filelist, ' has ', num2str(nfiles), ' files.']);\nfigcount = 0;\n% Processing txt files\nfor i = 1:nfiles\n% checking what kind of file (single or mapping) it is\nfilename = char(filenames(i));\nfprintf('%s: single file\n', filename);\n[x,y] = textread(filename,'%f%f','headerlines',1); % Reads in txt file\ndata = ...
1       #########################################\n## visualization of simulation results ##\n#########################################\n\npar(bg="transparent")\n#-------------------------------------------------------------------------------\n# stacked bar plot of spreads and fees\nplot_spreads_fees <- function(){\npar(bg="transparent")\n#aggregate spreads and fees\n#take 95% quantile\nfees <- aggregate(step3$costs, by=list(step3$date),\nfunction(x)quantile(x,probs = 0.95))\n\nspreads <

In [50]:
if DATE == "20231021":
    # ADDITIONAL PREPROCESSING OF DESCRIPTIONS

    # remove parentheses
    df_long.Description = df_long.Description.str.replace(r"\(.+?\)", "", regex=True)

    # remove URL
    df_long.Description = df_long.Description.str.replace(
        r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""",
        "",
        regex=True,
    )

    # ADDITIONAL PREPROCESSING OF CODE
    df_long.code_script = df_long.code_script.str.replace(
        r"#", "", regex=True
    ).str.replace(r"\n", " ", regex=True)
    df_long.loc[df_long.type_script == "m", "code_script"] = df_long.loc[
        df_long.type_script == "m", "code_script"
    ].str.replace(r"\%", " ", regex=True)

    # remove the same sign repeated more than 4 times
    df_long.code_script = df_long.code_script.str.replace(
        r"(.)\1{4,}", r"\1", regex=True
    )
    df_long.code_script = df_long.code_script.str.replace(
        "\s{2,}", "", regex=True
    ).str.strip()

In [52]:
df_long["Q_ID"] = df_long.index

df_long.to_csv(f"../../data/preprocessed/Quantlet/{DATE}/full_{DATE}.csv", index=False)

In [53]:
df_long["url"] = df_long.progress_apply(combine_url, axis=1)

100%|██████████| 4828/4828 [00:00<00:00, 40030.01it/s]


In [54]:
df_long[["Description", "url"]].to_csv(
    f"../../data/preprocessed/Quantlet/{DATE}/Description_annotation_{DATE}.csv",
    index=True,
)

In [55]:
df_long[df_long.Description == ""]

,folder_name,code_script,type_script,script_name,Quantlet,Description,Keywords,Authors,scr_n,description_len,description_n_words,repo,code_len,new_len,new_len2,Q_ID,url


In [56]:
# SPLIT THE DATA GROUP QUANTLET

labelled_qs, test_qs = train_test_split(
    list(df_long.Quantlet.unique()), test_size=0.1, random_state=RS
)
train_qs, val_qs = train_test_split(labelled_qs, test_size=0.1, random_state=RS)


train = df_long[df_long["Quantlet"].isin(set(train_qs))].reset_index(drop=True)
val = df_long[df_long["Quantlet"].isin(set(val_qs))].reset_index(drop=True)
test = df_long[df_long["Quantlet"].isin(set(test_qs))].reset_index(drop=True)

save_datasets(train=train, val=val, test=test, DATE=DATE, RS=RS)

(3945, 17)
r     0.462357
m     0.283397
py    0.254246
Name: type_script, dtype: float64
(391, 17)
r     0.511509
m     0.255754
py    0.232737
Name: type_script, dtype: float64
(492, 17)
r     0.436992
m     0.323171
py    0.239837
Name: type_script, dtype: float64
(3945, 17)
r     1824
m     1118
py    1003
Name: type_script, dtype: int64
(391, 17)
r     200
m     100
py     91
Name: type_script, dtype: int64
(492, 17)
r     215
m     159
py    118
Name: type_script, dtype: int64
